In [138]:
#Imports
import pandas as pd
import numpy as np
from numpy.linalg import pinv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import torch
import torch.nn as nn
from torch.autograd import Variable

In [139]:
#Load Data
fname = 'iris.csv'
print('Loading data from file: {}'.format(fname))
data = pd.read_csv(fname).values
# print(data[:5])
np.random.shuffle(data)
print('{} samples loaded with {} features'.format(data.shape[0], data.shape[1]-1))
# print(data[:5])
# print(data.shape)

Loading data from file: iris.csv
150 samples loaded with 4 features


In [140]:
#Split Labels and Features
feats = data[:,:-1]
labels_raw = data[:, -1]
# print(feats[:5])
# print(labels[:5])
labels_set = set(labels_raw)
print('Labels are: {}'.format(labels_set))
labels2index_map = {label: ind for ind, label in enumerate(labels_set)}
print('Labels to index map: {}'.format(labels2index_map))
labels = np.array([labels2index_map[label] for label in labels_raw])
# print(labels[:5])
# print(labels_raw[:5])

Labels are: {'virginica', 'setosa', 'versicolor'}
Labels to index map: {'virginica': 0, 'setosa': 1, 'versicolor': 2}


In [141]:
#Scaling
print(feats[:5])
scaler = MinMaxScaler()
feats = scaler.fit_transform(feats)
print('Scaling Features Done')
print(feats[:5])

[[6.6 2.9 4.6 1.3]
 [6.7 2.5 5.8 1.8]
 [5.5 2.6 4.4 1.2]
 [5.8 2.7 5.1 1.9]
 [6.0 2.9 4.5 1.5]]
Scaling Features Done
[[0.63888889 0.375      0.61016949 0.5       ]
 [0.66666667 0.20833333 0.81355932 0.70833333]
 [0.33333333 0.25       0.57627119 0.45833333]
 [0.41666667 0.29166667 0.69491525 0.75      ]
 [0.47222222 0.375      0.59322034 0.58333333]]


/home/rajasekar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [142]:
train_X, test_X, train_y, test_y = train_test_split(feats, labels, test_size=0.2, random_state=42)
print('Divided into training and testing set')
# print(train_X[:5])
# print(train_y[:5])
# print(test_X[:5])
# print(test_y[:5])

Divided into training and testing set


In [143]:
#To Bipolar
def to_bipolar(y, labels2index_map):
    y_bipolar = np.ones((len(y), len(labels2index_map))) * -1
    for i, label in enumerate(y):
        y_bipolar[i, label] = 1
    return y_bipolar
train_y_bipolar = to_bipolar(train_y, labels2index_map)
test_y_bipolar = to_bipolar(test_y, labels2index_map)
# print(train_y[:5])
# print(to_bipolar(train_y[:5], labels2index_map))
print('Converted labels to bipolar')

Converted labels to bipolar


In [144]:
#Sigmoid
def sigmoid(data):
    result = 1 / (1 + np.exp(-1 * data))
    return result
# print(sigmoid(np.array([-1, 0, 1])))

In [145]:
#Batch ELM
nHidden = 10

input_dim = train_X.shape[1]
hidden_dim = nHidden
output_dim = train_y_bipolar.shape[1]

i2h = np.random.uniform(-1,1,(input_dim, hidden_dim))

In [146]:
#ELM Train
H = sigmoid(np.dot(train_X, i2h))
h2o = np.dot(pinv(H), train_y_bipolar)
print(h2o[:5])

#ELM Predict
H = sigmoid(np.dot(test_X, i2h))
y_pred = np.dot(H, h2o)
print(y_pred[:5])
y_pred = np.array([np.argmax(y_pred[i,:]) for i in range(len(y_pred))])
print(y_pred)
print((test_y))#>0)*1)

[[  91.12603529   21.3422548  -111.8557072 ]
 [-177.28266371  452.54893475 -278.93034761]
 [  24.03478466 -205.27133861  182.38275925]
 [   6.76470279   81.70403368  -88.74056053]
 [ 293.06916878  -15.75988761 -276.54524593]]
[[ 1.67832788 -1.04150422 -1.6367777 ]
 [-0.73239455 -0.899748    0.63218459]
 [-0.50401053 -0.84404674  0.34886642]
 [-0.85459467  0.93320433 -1.07835141]
 [-0.55472898 -0.77679943  0.33195204]]
[0 2 2 1 2 2 1 2 0 0 1 2 1 0 1 2 2 0 2 2 1 1 1 1 2 1 1 2 1 1]
[0 2 2 1 2 2 1 2 0 0 1 2 1 0 1 2 2 0 2 2 1 1 1 1 2 1 1 2 1 1]


In [147]:
print(classification_report(test_y, y_pred))
print(accuracy_score(test_y, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00        13
          2       1.00      1.00      1.00        12

avg / total       1.00      1.00      1.00        30

1.0
